# About this document

This is a non-exhaustive tutorial on NGSolve. It should help you to get started with NGSolve or to refresh your memory. For all the topics discussed here there is much more material, see below.

Purpose of this tutorial:
 * Introduction to some very basic concepts in Netgen/NGSolve
 * Experience with NGSolve (Visualization, vectors, ..) and using the documentation 

Further Material:
 * [official jupyter tutorials of NGSolve](https://ngsolve.org/docu/latest/i-tutorials/index.html)
 * [jupyter-based course on Discontinuous Galerkin methods using NGSolve](http://num.math.uni-goettingen.de/~lehrenfeld/sections/pubs_src/DG-NGS-Hasselt.tar.gz)
 * [NGSolve Tutorial based on applications by J. Schöberl](https://ngsolve.org/iFEM/iFEM.zip)
 * [Crash course on FEM with NGSolve](http://num.math.uni-goettingen.de/~lehrenfeld/sections/pubs_src/crash_course_fem.tar.gz)
 

Import NGSolve.
For the visualization we use `pyvista` if available, otherwise we use the netgen GUI (assuming it can be spawned):

In [ ]:
from ngsolve import *
import netgen.geom2d
from draw import Draw

# Geometries and meshes in 2D

The functionality for describing 2D Geometries with Netgen from python are in the `geom2d` library. 

To check the available objects in that library you may try
```
dir(netgen.geom2d)
``` 

In [ ]:
dir(netgen.geom2d)

To find out more information about an object or a function you may use the contextual help (jupyterlab) or

In [ ]:
help(netgen.geom2d.SplineGeometry)

*Netgen* stores and handles meshes while *NGSolve* works with the mesh when computing PDE approximations. In *NGS-Py* there are two different type of "meshes": 
 * *Netgen*-meshes that store and handle geometry, nodes, etc...
 * *NGSolve*-meshes which are a wrapper around the *Netgen*-mesh that offer more top-level information that are important for discretization (boundary conditions, loop over elements, ...). We notice that an *NGSolve*-mesh `mesh` has an object `mesh.ngmesh` which is the underlying *Netgen*-mesh. For  visualization we will always use only the *NGSolve*-mesh.

In [ ]:
from netgen.geom2d import unit_square
mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))
Draw(mesh)

## Generating a geometry

Most geometries in 2D can easily be described with Splines. 

We will only import the `SplineGeometry` class from the `geom2d`:

In [ ]:
from netgen.geom2d import SplineGeometry
geo = SplineGeometry()

Now we can use one of the predefined objects (Rectangle,Circle) or generate our own geometry with lines or rational splines of 2nd order.

### Predefined geometries
There are some simple predefined geometries (check `dir(geo)`), e.g. `rectangle` and `circle`.

In [ ]:
geo_rect = SplineGeometry()
geo_rect.AddRectangle((-1,-1),(1,1),bc="rectangle")
#print(dir(geo))

Every `SplineGeometry` can call the mesh generator to obtain a *Netgen*-mesh:

In [ ]:
ngmesh_rect = geo_rect.GenerateMesh(maxh=0.1)
ngsmesh_rect = Mesh(ngmesh_rect)
Draw(ngsmesh_rect)

In [ ]:
geo_circle = SplineGeometry()
geo_circle.AddCircle((0,0),0.5,bc="circle")
ngmesh_circle = geo_circle.GenerateMesh(maxh=0.4)
ngsmesh_circle = Mesh(ngmesh_circle)
ngsmesh_circle.Curve(3)
Draw(ngsmesh_circle)  #in the GUI version switch to "solution" and increase the subdivision manually

## Using lines and splines
* define a new geometry `geo`
* write a list of points 
* add them to geometry `geo`.

In [ ]:
geo_curved = SplineGeometry()

pnts =[(0,0),
       (1,0),
       (1,0.5),
       (1,1),
       (0.5,1),
       (0,1)]

p1,p2,p3,p4,p5,p6 = [geo_curved.AppendPoint(*pnt) for pnt in pnts]

Then we define the curves which define our geometry and add them to the geometry using `Append`.

<img src="img/curved.png" width="320" height="320" align="center"/>

In [ ]:
curves = [[["line",p1,p2],"bottom"],
          [["line",p2,p3],"right"],
          [["spline3",p3,p4,p5],"curve"],
          [["line",p5,p6],"top"],
          [["line",p6,p1],"left"]]

[geo_curved.Append(c,bc=bc) for c,bc in curves]

In [ ]:
ngmesh_curved = geo_curved.GenerateMesh(maxh=0.2)
ngsmesh_curved = Mesh(ngmesh_curved)
ngsmesh_curved.Curve(3)
Draw(ngsmesh_curved) #in the GUI version switch to "solution" and increase the subdivision manually

## Hands-On-Task 1:
1. Generate a mesh of a triangular domain.
2. (If time permits) Make up your own interesting geometry.

<img src="img/triangle.png" width="320" height="320" align="center"/>

In [ ]:
#TODO

# FEM

## `FESpace`s  and `GridFunction`s

In [ ]:
mesh = ngsmesh_circle
print("number of elements:", mesh.ne)
Draw(mesh)

A finite element space defines a set of basis functions on the mesh:

In [ ]:
fes = H1(mesh, order=1)
fes.ndof

A `GridFunction` is the connection between the basis functions of a finite element space and a linear algebra vector $\underline{u} \in \mathbb{R}^N$:
$$
u_h(x) = \sum_{i=1}^N \underline{u}_i \varphi_i(x)
$$

In [ ]:
gfu = GridFunction(fes)
gfu.vec[:] = 0; gfu.vec[16] = 1
Draw(0.25*gfu,mesh,"gfu")

### Hands-On-Task 2:
1. Inspect the following finite element spaces:
`H1`, `L2`, `HDiv`, `HCurl`
What can you say about the basis functions (and hence about the space) w.r.t. dimension, continuity, degrees of freedom?

2. Consider `fes=FESpace(type="nonconforming",mesh=ngsmesh)`. 
What are $(T,V_T,\Psi_T)$ (domain, function space, functionals) in the definition of this finite element? 

3. Interpolate the function $x^2$ on this space with `GridFunction`'s member function `Set(..)`.

In [ ]:
#TODO

## discretized PDE solutions

Example here: Weak form of the Poisson problem Dirichlet boundary conditions: 
Find $u \in V = H_0^1(\Omega)$ such that
$$
A(u,v) = \int_\Omega \nabla u \cdot \nabla v = \int_\Omega f v = f(v)
$$
for all $v \in V = H_0^1(\Omega)$.

Restriction of the weak formulation to $V_h$:
$$
\text{find} \, u_h \in V_h : \quad A(u_h,v_h) = f(v_h) \quad \forall \, v_h \in V_h
$$

Equivalent linear system:
$$
 \underline{A} \cdot \underline{u} = \underline{f }
$$
with $\underline{A}_{ij} = A(\varphi_j,\varphi_i)$, $\underline{f}_{i} = f(\varphi_i)$. 

In [ ]:
mesh = ngsmesh_circle
#fes = FESpace(type="nonconforming",
#              mesh=mesh,dirichlet="circle")
fes = H1(mesh=mesh,dirichlet="circle")
gfu = GridFunction(fes)

In [ ]:
u,v = fes.TnT()

In [ ]:
# specify the forms by means of trial and test-functions:
a = BilinearForm(fes)
f = LinearForm(fes)
a += SymbolicBFI (grad(u)*grad(v))
f += SymbolicLFI (10*v)

In [ ]:
# compute the matrix and right hand side vector
a.Assemble()
f.Assemble()

### Dirichlet boundary conditions

In [ ]:
gfu.Set(x,definedon=mesh.Boundaries("circle"))

If $A=$ `a.mat` is the matrix just assembled, then we want to solve for 

$$
  A (u_0 + u_D) = f \quad \Rightarrow \quad A u_0 = f - A u_D
$$

or

$$
  \left( \begin{array}{cc} A_{FF} & A_{FD} \\ A_{DF} & A_{DD} \end{array} \right) \left( \begin{array}{c} u_{0,F} \\ 0 \end{array} \right) = \left( \begin{array}{c} {f}_F \\ {f}_D \end{array} \right) - \left( \begin{array}{cc} A_{FF} & A_{FD} \\ A_{DF} & A_{DD} \end{array} \right) \left( \begin{array}{c} u_{D,F} \\ u_{D,D} \end{array} \right)
$$

where we have block partitioned using free dofs ($F$) and dirichlet dofs ($D$) as if they were numbered consecutively (which may not be the case in  practice) for ease of presentation.  The first row gives

$$
A_{FF} u_{0,F} = f_F - [A u_D]_F.
$$

Since we have already constructed $u_D$, we need to perform 
these next steps:

- Set up the right hand side from $f$ and $u_D$.
- Solve a linear system which involves only $A_{FF}$.
- Add solution: $u = u_0 + u_D$.

In [ ]:
# solve the linear system
f.vec.data -= a.mat * gfu.vec
gfu.vec.data += a.mat.Inverse(freedofs=fes.FreeDofs()) * f.vec
Draw (gfu,mesh,"gfu")

The linear system:

In [ ]:
import scipy.sparse as sp
import matplotlib.pyplot as plt

A=sp.csr_matrix(a.mat.CSR())
plt.figure(figsize=(7,7))
plt.spy(A); plt.show()

In [ ]:
print(fes.FreeDofs())

### Hands-On-Task 3:
1. Solve the PDE problem
$$
 - \varepsilon \Delta u + u = 0 \quad \text{ on } \Omega = \operatorname{conv}((-1,0),(1,0),(0,1))
$$
with $u = 1$ on $\{ y = 0\}$ and $\nabla u \cdot n = 0$ on the remainder of the boundary and $\varepsilon = 0.01$.

2. What do you observe? What do you expect for $\varepsilon =1$ and $\varepsilon \to 0$? (Try it out).

# Linear solvers

In [ ]:
from ngsolve import solvers
from ngsolve.solvers import CG

Diagonal preconditioner with filter for `FreeDofs`:

In [ ]:
pre = a.mat.CreateSmoother(fes.FreeDofs())

In [ ]:
gfu.vec.data += CG(mat=a.mat,rhs=f.vec,pre=pre,
                  printrates=True, initialize=True)

## Hands-On-Task 4:
1. Write a Richardson iteration scheme for the iterative solution of $C^{-1} A \cdot x = C^{-1} f$ where $C = \operatorname{diag}(A)$.

* $x^0$
* Loop over k until $k=1000$ (add other stopping criteria if you want)
  * $r^k = f - A \cdot x^k$
  * $s^k = C^{-1} r^k$
  * $x^{k+1} = x^k + \omega s^k$
  
How do you need to choose $\omega$ to obtain a converging scheme?

Hint:
 * The Jacobi preconditioner $C^{-1}$ can be obtained with
 ```
 c = a.mat.CreateSmoother()
 ```
 * You can create vectors of the same type as `gfu.vec` with
 ```
 res = gfu.vec.CreateVector()
 ```
 * Make sure to set Dirichlet dofs to zero when measuring the residual norm $\Vert f - A \cdot x^k \Vert$. For that use
 ``` 
 res.data = Projector(fes.FreeDofs(),True) * res
 ```

In [ ]:
gfu.Set(x,definedon=mesh.Boundaries("circle"))
#TODO